In [1]:
# !pip install bayesian-optimization

In [ ]:
import deepquantum as dq
import deepquantum.photonic.circuit as circuit
import matplotlib.pyplot as plt
import numpy as np

from deepquantum.optimizer import *
from deepquantum.photonic.decompose import *
from scipy.stats import unitary_group

np.set_printoptions(precision=8, floatmode='fixed', suppress=True) # to make the print info aligned

N = 8
u8x8 = unitary_group.rvs(N)
decomp_rlt = UnitaryDecomposer(u8x8, "cssr").decomp()
mzi_info = decomp_rlt[0]

def zero_init(mzi_info):
    for i in range(len(mzi_info['MZI_list'])):
        mzi_info['MZI_list'][i][2] = 0.0
        mzi_info['MZI_list'][i][3] = 0.0
    mzi_info['phase_angle_ori'] = np.zeros(N)
    mzi_info['phase_angle'] = np.zeros(N)

def bar_securing(mzi_info):
    adjustable_ids = []
    for i in range(len(mzi_info['MZI_list'])):
        if i not in adjustable_ids:
            mzi_info['MZI_list'][i][-1] = np.pi
            mzi_info['MZI_list'][i][-2] = np.pi

def xx_planting(mzi_info):
    for i in [20, 24]:
        mzi_info['MZI_list'][i][-1] = np.pi/2

def yy_planting(mzi_info):
    for i in [20, 24]:
        mzi_info['MZI_list'][i][-1] = np.pi/2
        mzi_info['MZI_list'][i][-2] = np.pi

def zz_planting(mzi_info):
    for i in [20, 24]:
        mzi_info['MZI_list'][i][-1] = np.pi
        mzi_info['MZI_list'][i][-2] = np.pi

def zx_planting(mzi_info):
    mzi_info['MZI_list'][20][-1] = np.pi
    mzi_info['MZI_list'][20][-2] = np.pi
    mzi_info['MZI_list'][24][-1] = np.pi/2


def trial_planting(mzi_info,angle_list):
    change_list = [[1,-1],
                   [4,-1],
                   [5,-1]]
    for i in range(len(angle_list)):
        a,b = change_list[i]
        mzi_info['MZI_list'][a][b] = angle_list[i]

zero_init(mzi_info)
bar_securing(mzi_info)
zz_planting(mzi_info)


def post_selection(rlt_strkey):
    # print(rlt_strkey)
    label_list = ['|00101000>', '|00100100>', '|00011000>', '|00010100>']
    value = np.zeros(4)
    for i in range(4):
        try:
            value[i] = rlt_strkey[label_list[i]]
        except:
            pass
    return value


def estimate_energy(post_selection_rlt,type='ZZ'):
    norm = post_selection_rlt/(post_selection_rlt.sum()+1e-16)
    if type=='ZZ': # ZZ basis
        return norm[0]+norm[3]-norm[1]-norm[2]
    if type=='ZI': # ZI basis
        return norm[0]+norm[1]-norm[2]-norm[3]
    if type=='IZ': # IZ basis
        return -norm[0]-norm[1]+norm[2]+norm[3]

def compute_process(mzi_info):
    cir = circuit.QumodeCircuit(nmode=8,init_state=[0, 0, 0, 1, 0, 1, 0, 0], name="test", cutoff = 3, basis = True) # basis=True, using state list
    for info in mzi_info['MZI_list']:
        cir.ps(inputs=info[2],wires=[info[0]])
        cir.bs_theta(inputs=np.pi/4,wires=[info[0],info[1]])
        cir.ps(inputs=info[3],wires=[info[0]])
        cir.bs_theta(inputs=np.pi/4,wires=[info[0],info[1]])
    rlt = cir(is_prob=True)
    rlt_strkey = dict()
    for key in rlt.keys():
        rlt_strkey[str(key)] = np.abs(rlt[key].detach().numpy().squeeze())**2

    # rltm = cir.measure(shots=10000)[0]
    # rltm_strkey = dict()
    # for key in rltm.keys():
    #     rltm_strkey[str(key)] = rltm[key]
    # return post_selection(rltm_strkey)

    return post_selection(rlt_strkey)

def estimate_eigval(angle_list):
    # to minimize
    zero_init(mzi_info)
    bar_securing(mzi_info)
    trial_planting(mzi_info,angle_list)

    zz_planting(mzi_info)
    post_selection_rlt = compute_process(mzi_info)
    e_zz = estimate_energy(post_selection_rlt)

    xx_planting(mzi_info)
    post_selection_rlt = compute_process(mzi_info)
    e_xx = estimate_energy(post_selection_rlt)

    yy_planting(mzi_info)
    post_selection_rlt = compute_process(mzi_info)
    e_yy = estimate_energy(post_selection_rlt)

    return (1 + e_zz - e_xx - e_yy) / 2

    # zz_planting(mzi_info)
    # post_selection_rlt = compute_process(mzi_info)
    # e_zi = estimate_energy(post_selection_rlt,type='ZI')

    # xx_planting(mzi_info)
    # post_selection_rlt = compute_process(mzi_info)
    # e_ix = estimate_energy(post_selection_rlt,type='IZ')

    # zx_planting(mzi_info)
    # post_selection_rlt = compute_process(mzi_info)
    # e_zx = estimate_energy(post_selection_rlt,type='ZZ')

    # return (1 + e_zi + e_ix - e_zx) / 2


def target_function(angle_list):
    # to maximize
    return -estimate_eigval(angle_list)

def void_target():
    return -1

In [3]:
# 测试BO算法
print("="*50)
print("运行BO")
param_init = {'t1':0, 't2':0, 't3':0}
bo_optimizer = OptimizerBayesian(target_func=void_target,param_init=param_init,random_state=0)

print("轮次","\t","参数值","\t"," "*31,"目标值")
for _ in range(100):
    # 待片上计算的参数
    p1 = bo_optimizer.param_suggest()
    # 模拟芯片计算过程，之后需要替换成接口
    f1 = -estimate_eigval(p1.tolist()) #  BO 内置用法是最大化目标；但是接下来打印时再添一个符号即可
    # 测试结果传回，更新param_dict
    bo_optimizer.param_register([p1],[f1])
    f1_to_print = -f1
    if bo_optimizer.best_target == f1:
        mark = "*"
    else:
        mark = " "
    if f1_to_print<0:
        print(bo_optimizer.iter,"\t",p1,"\t",f"{f1_to_print:8.10f} ",mark)
    else:
        print(bo_optimizer.iter,"\t",p1,"\t ",f"{f1_to_print:8.10f} ",mark)

print("BO结果：",estimate_eigval(list(bo_optimizer.best_param_dict.values())))

运行BO
轮次 	 参数值 	                                 目标值
1 	 [3.44829694 4.49366732 3.78727399] 	  0.2558570575  *
2 	 [0.04130112 0.38026479 0.23805479] 	 -0.1574870785  *
3 	 [0.04864806 0.38761765 0.24540173] 	 -0.1906562783  *
4 	 [0.13037826 0.46941791 0.32712838] 	 -0.3973004700  *
5 	 [0.21688704 0.63756850 0.19842172] 	 -0.8221395275  *
6 	 [0.33112159 0.82144763 0.12905419] 	 -0.9128353694  *
7 	 [1.27243923 5.69805286 4.11127548] 	  1.7828855449   
8 	 [0.00000000 1.44800809 0.00000000] 	  0.0000000000   
9 	 [0.86063626 0.51379958 0.00000000] 	 -0.9891131998  *
10 	 [1.25513427 1.08474693 0.00000000] 	 -0.9998541182  *
11 	 [1.67915859 0.48457441 0.00000000] 	 -0.7171892080   
12 	 [1.24502626 0.83484519 0.65709919] 	 -0.8507857277   
13 	 [1.89141548 1.37895907 0.40890267] 	 -0.9842004371   
14 	 [1.04620301 0.00453775 1.19135049] 	  0.9759387713   
15 	 [1.24131709 1.70948440 0.68018922] 	 -0.9023975490   
16 	 [1.68061410 2.08870537 0.00000000] 	 -0.9958886909   
17 	 [2.07026

In [5]:
# 测试SPSA算法
print("="*50)
print("运行SPSA")
param_init = {'t1':np.random.randn(), 't2':np.random.randn(), 't3':np.random.randn()}
spsa_optimizer = OptimizerSPSA(target_func=void_target,param_init=param_init,random_state=0)

print("轮次","\t","参数值","\t"," "*31,"目标值")
for _ in range(200):
    # 待片上计算的两组参数
    p1,p2 = spsa_optimizer.param_suggest()
    # 以下两行模拟芯片计算过程，之后需要替换成接口
    f1 = estimate_eigval(p1.tolist())
    f2 = estimate_eigval(p2.tolist())
    # 测试结果传回，更新param_dict
    spsa_optimizer.param_register([p1,p2],[f1,f2])
    # spsa_optimizer.param_register(np.array([p1,p2]),np.array([f1,f2]))


    if f1<0:
        print(spsa_optimizer.iter,"\t",p1,"\t",f"{f1:8.10f} ")
    else:
        print(spsa_optimizer.iter,"\t",p1,"\t ",f"{f1:8.10f} ")
    if f2<0:
        print(spsa_optimizer.iter,"\t",p2,"\t",f"{f2:8.10f} ")
    else:
        print(spsa_optimizer.iter,"\t",p2,"\t ",f"{f2:8.10f} ")
    # print(spsa_optimizer.hyperparam['c']/(1+spsa_optimizer.iter)**spsa_optimizer.hyperparam['gamma'])

    if (f1<-0.92) and (f2<-0.92):
        # print("!!!进入步长减小阶段!!!")
        spsa_optimizer.hyperparam['c'] = 0.001
    elif (f1<-0.999) and (f2<-0.999):
        spsa_optimizer.hyperparam['c'] = 1e-4

print("SPSA结果：",estimate_eigval(list(spsa_optimizer.best_param_dict.values())))

运行SPSA
轮次 	 参数值 	                                 目标值
1 	 [ 0.16650654  0.22218104 -0.60731607] 	  0.7674033344 
1 	 [ 0.14650654  0.24218104 -0.58731607] 	  0.7503614604 
2 	 [ 0.16233125  0.22635633 -0.60314078] 	  0.7637734945 
2 	 [ 0.14368352  0.24500406 -0.58449305] 	  0.7480159108 
3 	 [ 0.14059791  0.23019017 -0.59930693] 	  0.7833624849 
3 	 [ 0.15849740  0.24808967 -0.58140744] 	  0.7197535700 
4 	 [ 0.15536077  0.24495304 -0.56715717] 	  0.7103299009 
4 	 [ 0.17274766  0.26233993 -0.58454406] 	  0.6856967771 
5 	 [ 0.17832027  0.25091312 -0.57311725] 	  0.6769788253 
5 	 [ 0.16132085  0.26791254 -0.59011666] 	  0.7023762751 
6 	 [ 0.18422808  0.26169455 -0.56720944] 	  0.6448994292 
6 	 [ 0.16753883  0.24500531 -0.58389868] 	  0.7139137643 
7 	 [ 0.20083142  0.26186648 -0.55060609] 	  0.5924996344 
7 	 [ 0.18440000  0.27829790 -0.56703751] 	  0.6202536419 
8 	 [ 0.19132460  0.25516200 -0.54390161] 	  0.6063431266 
8 	 [ 0.20753590  0.27137330 -0.56011291] 	  0.5838876895 
9 

In [6]:
# 测试Fourier级数方法
print("="*50)
print("运行Fourier级数法")
param_init = {'t1':np.random.randn(), 't2':np.random.randn(), 't3':np.random.randn()}
# param_init = dict(zip(param_init.keys(),np.random.randn(3)))
fourier_optimizer = OptimizerFourier(target_func=void_target,param_init=param_init,random_state=0,order=3,lr=0.1)
print("轮次","\t","参数值","\t"," "*31,"目标值")
for _ in range(50):
    # 待片上计算的两组参数
    param_array = fourier_optimizer.param_suggest()
    # 模拟芯片计算过程，之后需要替换成接口
    target = np.zeros(len(param_array))
    for i in range(len(param_array)):
        target[i] = estimate_eigval(param_array[i])

    # 测试结果传回，更新param_dict
    fourier_optimizer.param_register(param_array,target)
    p1 = np.array(list(fourier_optimizer.best_param_dict.values()))
    f1 = fourier_optimizer.best_target
    if f1<0:
        print(fourier_optimizer.iter,"\t",p1,"\t",f"{f1:8.10f} ")
    else:
        print(fourier_optimizer.iter,"\t",p1,"\t ",f"{f1:8.10f} ")

print("Fourier结果：",estimate_eigval(list(fourier_optimizer.best_param_dict.values())))

运行Fourier级数法
轮次 	 参数值 	                                 目标值
1 	 [0.15650654 0.23218104 0.00000000] 	 -0.9970601675 
2 	 [0.28570363 0.35736381 0.00000000] 	 -0.9996698515 
3 	 [0.45135446 0.53716853 0.00000000] 	 -0.9998663196 
4 	 [0.56920008 0.63571517 0.00000000] 	 -0.9999762892 
5 	 [0.66683487 0.71861761 0.00000000] 	 -0.9999945995 
6 	 [0.74784723 0.78859492 0.00000000] 	 -0.9999985221 
7 	 [0.81518171 0.84777687 0.00000000] 	 -0.9999995268 
8 	 [ 0.89759790  0.89773209 -0.00671025] 	 -0.9999999975 
9 	 [ 0.89759790  0.89773209 -0.00671025] 	 -0.9999999975 
10 	 [ 0.89759790  0.89773209 -0.00671025] 	 -0.9999999975 
11 	 [ 0.89759790  0.89773209 -0.00671025] 	 -0.9999999975 
12 	 [ 0.89759790  0.89773209 -0.00671025] 	 -0.9999999975 
13 	 [ 0.89759790  0.89773209 -0.00671025] 	 -0.9999999975 
14 	 [ 0.89759790  0.89773209 -0.00671025] 	 -0.9999999975 
15 	 [1.07586069 1.08539410 0.00000000] 	 -0.9999999983 
16 	 [1.09027383 1.09882557 0.00000000] 	 -0.9999999989 
17 	 [1.10272695